# RAG

- HyDE 와 비교하기 위해서 기본 설정으로 검색해본다.

# 0. Setup

In [1]:
!pip -q install -U boto3 awscli langchain pypdf

In [31]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.document_loaders import TextLoader
import langchain

In [3]:
profile_name = None
region = 'us-east-1'

In [4]:
# modelId = 'anthropic.claude-instant-v1'
modelId = 'anthropic.claude-v2'

In [5]:
session = boto3.Session(
    profile_name=profile_name,
    region_name=region,
)
bedrock = session.client(service_name='bedrock-runtime')

## Embeddings

- bedrock embedding model - amazon.titan-embed-text-v1
- 일반적으로는 SOTA 인 baai 의 [bge](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) 씀.

In [8]:
llm = Bedrock(
    model_id=modelId,
    client=bedrock,
    model_kwargs={
        'max_tokens_to_sample': 1024
    },
)
embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock,
)

In [44]:
langchain.debug = False

# 1. Usage

- AWS IoT Provisioning Whitepaper [다운로드](https://docs.aws.amazon.com/pdfs/whitepapers/latest/device-manufacturing-provisioning/device-manufacturing-provisioning.pdf#device-manufacturing-provisioning)
- 간접적인 질문을 통해 HyDE 사용

In [45]:
loader = PyPDFLoader("iot.pdf")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = loader.load_and_split(text_splitter=text_splitter)

In [46]:
len(texts)

36

In [47]:
texts[0]

Document(page_content='Device Manufacturing and \nProvisioning with X.509 \nCertiﬁcates in AWS IoT Core\nAWS Whitepaper', metadata={'source': 'iot.pdf', 'page': 0})

In [48]:
%%time

docsearch = Chroma.from_documents(texts, embeddings)

CPU times: user 359 ms, sys: 43.2 ms, total: 402 ms
Wall time: 12.1 s


In [49]:
retriever = docsearch.as_retriever(search_kwargs={"k": 5})
retriever.search_type

'similarity'

In [50]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [52]:
query = "What is the best way to provision device when I can not put certificates into my devices?"
llm_response = qa_chain(query)

In [55]:
print(llm_response['result'])

 Based on the provided context, the best way to provision devices when you cannot put certificates directly into the devices is to use AWS IoT Core's fleet provisioning. 

Key points:

- Fleet provisioning allows you to provision devices without having to put certificates directly on the devices. 

- With fleet provisioning, the device maker sets up a provisioning template in AWS IoT Core. Devices are then provisioned by having a trusted mobile app request temporary credentials from AWS IoT using the CreateProvisioningClaim API. 

- The temporary credentials are sent to the device over a secure connection like Bluetooth, WiFi, or USB. 

- The device then uses the temporary credentials to connect to AWS IoT Core where it can get its permanent credentials.

- The device maker never needs direct access to put certificates on devices. The whole process is handled through the provisioning template and API call from the trusted mobile app.

So in summary, if you can't put certificates direct

In [60]:
print(llm_response['source_documents'][0].page_content)

Device Manufacturing and Provisioning with X.509 
Certiﬁcates in AWS IoT Core AWS Whitepaper
Fleet provisioning by trusted user
Setup
Device makers that use the Trusted User ﬂow must develop and maintain a mobile application that uses 
the CreateProvisioningClaim  API. The ﬂeet provisioning template must be set up and maintained in 
AWS IoT Core by the device maker. Optionally, a pre-provisioning AWS Lambda function is recommended 
to provide additional authentication steps during the provisioning process.
Device logic
Devices must have the ability to accept temporary credentials over a secure connection such as Bluetooth 
Low Energy, WiFi, or USB. Devices must implement the logic necessary to publish and subscribe to ﬂeet 
provisioning MQTT topics, accept the permanent credentials, and write the credentials to secure storage.
20
